In [1]:
from pathlib import Path
from datetime import date
import json

from shapely.geometry import box, mapping


In [2]:
import sys
import os

# Get the path of the project root directory 
# Assumes you are running jupyter lab from inside the 'earth-intelligence-platform' directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to the system path so Python can find 'eintelligence'
if project_root not in sys.path:
    sys.path.append(project_root)
    
print(f"Project root added to path: {project_root}")

Project root added to path: /home/hegde/code/earth-api


In [ ]:
from eintelligence.data_prep.aoi import square_aoi
from orchestrator.workflow_manager import (
    DeforestationWorkflow, FloodWorkflow, TilingConfig, TrainingConfig
)

# central california
aoi = square_aoi(36.3, -120.7)
print(aoi["geometry"]["coordinates"][0][:2])  # check polygon corners

# munich, germany
# aoi = square_aoi(48.1351, 11.5820)
# print(aoi["geometry"]["coordinates"][0][:2])  # check polygon corners

/home/hegde/code/earth-api/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


((-120.64571141725517, 36.253876631735636), (-120.64300137551535, 36.34395516181293))


In [4]:
# aoi_geom = mapping(box(-120.7, 36.3, -120.3, 36.7))  # (minx, miny, maxx, maxy)
# aoi_geojson = {"type": "Feature", "geometry": aoi_geom, "properties": {}}


In [5]:
tiling_cfg = TilingConfig(bands_s2=("B02","B03","B04","B08"), tile_size=512, stride=256, max_cloud=20)
train_cfg  = TrainingConfig(batch_size=8, num_epochs=10, amp=True)

In [6]:
defor = DeforestationWorkflow(project_root, tiling_cfg, train_cfg, skip_to_pairing=True)

pairs_manifest = defor.build_data(
    aoi_geojson=aoi,
    start="2023-06-01",
    end="2023-09-01",
    region_name="central_ca"
)

Skipping to temporal pairing using existing tiled data...
pairs manifest: /home/hegde/code/earth-api/data/central_ca/pairs_manifest.json


/home/hegde/code/earth-api/orchestrator/workflow_manager.py:116: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda" and cfg.amp))


In [7]:
ckpt_path = Path(project_root) / "models" / "deforestation_resnet18.pt"
out_dir = Path(project_root) / "data" / "central_ca" / "pred_deforestation"

defor.run_deforestation_workflow(pairs_manifest, ckpt_path, out_dir, retrain=False)



Using existing checkpoint: /home/hegde/code/earth-api/models/deforestation_resnet18.pt, skipping training.


/home/hegde/code/earth-api/orchestrator/workflow_manager.py:284: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast(enabled=(device.type=="cuda")):


wrote 16 deforestation mask tiles -> /home/hegde/code/earth-api/data/central_ca/pred_deforestation
